In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from typing import Iterable, Tuple, List, Callable

In [2]:
def objective_func(x):
    return x * np.sin(10 * np.pi * x) + 2.0

In [68]:
CHROMOSOME_SIZE = 12
POPULATION_SIZE = 6
CROSSOVER_RATE = 0.8

x_min = -1
x_max = 2

6


x_range = np.linspace(x_min,x_max, 100)
y_range = objective_func(x_range)
plt.plot(x_range, objective_func(x_range))

In [145]:
def bin_to_float(bin_arr, a = 0, b = 1):
    
    arr_len = len(bin_arr)
    
    dec = bin_arr @ np.power(2, np.arange(arr_len))
    
    zero_one_range = dec / (2**arr_len)
    
    real_num = zero_one_range * (b - a) + a
    
    return real_num

def initialize_genotype_population(population_size, chromo_size):
    
    x = np.random.randint(low = 0, high = 2, size = (population_size, chromo_size), dtype=bool)
    
    return x
    

def genotype_to_phenotype(x, x_min=0, x_max=1):
    
    df = pd.DataFrame(x)
    
    return df.apply(bin_to_float, axis=1, raw=True, a=x_min, b=x_max)

In [132]:
def bin_to_float(bin_arr, a = 0, b = 1):
    
    arr_len = bin_arr.shape[-1]
    
    dec = bin_arr @ np.power(2, np.arange(arr_len))
    
    zero_one_range = dec / (2**arr_len)
    
    real_arr = zero_one_range * (b - a) + a
    
    return real_arr

In [146]:
def evaluate(x_phenotype):
    return x_phenotype.map(objective_func)

In [156]:
x = np.concatenate((np.random.randint(low=0, high=2,size=(4, 12),dtype=bool),np.array([[1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1]], dtype=bool)))
x

array([[False,  True, False,  True, False,  True, False, False, False,
        False, False,  True],
       [ True,  True,  True, False, False,  True,  True, False,  True,
        False,  True,  True],
       [ True,  True, False, False,  True,  True, False, False,  True,
         True,  True,  True],
       [False,  True,  True,  True,  True, False,  True,  True,  True,
         True,  True, False],
       [ True,  True, False,  True, False,  True,  True, False, False,
         True, False,  True]])

In [157]:
x_int = x.astype(int)

In [158]:
x_int

array([[0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1],
       [1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1],
       [1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0],
       [1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1]])

In [159]:
bin_to_float(x, -1, 2)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 5 is different from 12)

In [140]:
x_int = np.random.randint(0,2,size=(2,3,12),dtype=bool)

In [141]:
x_int

array([[[False, False, False,  True, False, False,  True, False, False,
         False,  True,  True],
        [False, False,  True,  True, False, False, False,  True,  True,
         False,  True,  True],
        [False,  True, False,  True, False,  True,  True,  True, False,
         False,  True,  True]],

       [[False,  True,  True, False, False,  True,  True,  True, False,
         False, False, False],
        [False,  True, False, False,  True,  True,  True,  True,  True,
         False, False, False],
        [False,  True, False,  True,  True, False, False,  True, False,
         False,  True, False]]])

In [142]:
bin_to_float(x_int)

array([[0.76757812, 0.84667969, 0.80712891],
       [0.05615234, 0.12158203, 0.28759766]])

In [149]:
x_genotype = initialize_genotype_population(POPULATION_SIZE, CHROMOSOME_SIZE)
x_genotype

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
        False,  True, False],
       [ True,  True, False,  True, False, False,  True,  True,  True,
         True, False, False],
       [ True,  True, False, False,  True, False, False, False, False,
         True, False, False],
       [ True,  True,  True,  True, False, False, False,  True, False,
         True, False, False],
       [False,  True,  True, False, False, False, False,  True,  True,
        False, False,  True],
       [False,  True, False, False,  True,  True, False,  True,  True,
        False,  True,  True]])

In [150]:
%%time
x_phenotype = genotype_to_phenotype(x_genotype, x_min, x_max)

CPU times: user 737 µs, sys: 110 µs, total: 847 µs
Wall time: 759 µs


In [151]:
%%time
evals = evaluate(x_phenotype)

CPU times: user 447 µs, sys: 0 ns, total: 447 µs
Wall time: 453 µs


In [78]:
def calc_relative_fittness(evals):
    return evals/evals.sum()
    

In [79]:
rel_fittness = calc_relative_fittness(evals)
rel_fittness

0    0.242354
1    0.156953
2    0.291445
3    0.011162
4    0.124136
5    0.173951
dtype: float64

In [80]:
def create_mating_pool(genotype_arr, prob_arr):
    indeces = np.random.choice(np.arange(len(prob_arr)), size=len(genotype_arr), replace=True, p=prob_arr)
    return genotype_arr[indeces]

In [81]:
mating_pool = create_mating_pool(x_genotype, rel_fittness)

In [82]:
mating_pool

array([[False, False,  True,  True,  True,  True, False,  True, False,
        False, False, False],
       [ True, False, False,  True,  True,  True, False,  True, False,
         True, False,  True],
       [False, False,  True,  True,  True,  True, False,  True, False,
        False, False, False],
       [ True, False, False,  True,  True,  True, False,  True, False,
         True, False,  True],
       [ True, False, False,  True,  True,  True, False,  True, False,
         True, False,  True],
       [False, False, False, False,  True, False,  True, False, False,
         True, False, False]])

In [83]:
def single_point_crossover(x: List):
    
    x0 = x[0]
    x1 = x[1]
    
    point = np.random.randint(low = 1, high = x.shape[1])
    
    x1[:point], x0[:point] = x0[:point], x1[:point].copy()
    
    return x

In [84]:
def flip_bit_mutate(x: np.array, mutation_rate: float = 0.1):
    
    probs = np.random.rand(x.shape[0], x.shape[1])
    
    mutat_mask = probs < mutation_rate
    
    x_mutated = x.copy()
    
    x_mutated[mutat_mask] = ~x_mutated[mutat_mask]
    
    return x_mutated
    

In [85]:
def split_to_chunks(x: Iterable, n: int = 2):
    '''
    splits x array into evenly sized chunks. 
    
    x: array to split into chunks.
    n: number of elements in chunks.
    '''
    assert len(x) % n == 0, 'length of x should be divisible on n with remainder equal to 0.'
    
    for i in range(0, len(x), n):
        yield x[i:i + n]  

In [86]:
def create_next_generation(x: Iterable, splitter: Callable, crossover: Callable, mutate: Callable, crossover_rate: float = 0.5, mutation_rate: float = 0.4):
    '''
    Performs crossover of individuals.
    
    x: mating pool of individuals to perform crossover on (2d array). dim0 - is individuals, dim1 - genes;
    c_rate: crossover rate. number bewteen 0 and 1. probability that given two individuals perform a crossover. 
    if two individuals do not perform a crossover according to probability c_r, their off-spring are themselves. 
    
    '''
    new_generation = []
    
    np.random.shuffle(x)
    
    chunks = splitter(x)

    for ch in chunks:
        
        if np.random.rand() < crossover_rate:
            ch = crossover(ch)
            
        ch = mutate(ch, mutation_rate)
            
        new_generation.extend(ch)
            
    return np.array(new_generation)

In [108]:
current_gen = create_next_generation(mating_pool, split_to_chunks, single_point_crossover, flip_bit_mutate, crossover_rate=0.8, mutation_rate=0.6)

In [109]:
current_gen.astype(int)

array([[1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0],
       [0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0],
       [1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1]])

In [92]:
mating_pool.astype(int)

array([[1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1],
       [1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1],
       [0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1]])

In [36]:
threshold = 0.5

In [24]:
probs = np.random.rand(x.shape[0], x.shape[1])
probs

array([[0.93044014, 0.91084093, 0.52710114, 0.54962189, 0.2810227 ,
        0.06790312, 0.79672629, 0.07770015, 0.74706906, 0.89533534,
        0.9813172 , 0.61324398],
       [0.08070496, 0.6997923 , 0.29712234, 0.67873172, 0.34556672,
        0.29118508, 0.17965867, 0.56863655, 0.58720649, 0.08113324,
        0.41847542, 0.63251462],
       [0.78848199, 0.71696768, 0.30451469, 0.97284076, 0.06229457,
        0.76860844, 0.99326829, 0.00416695, 0.76762551, 0.5484852 ,
        0.1392219 , 0.21698107]])

In [25]:
mutat_mask = probs < threshold
mutat_mask

array([[False, False, False, False,  True,  True, False,  True, False,
        False, False, False],
       [ True, False,  True, False,  True,  True,  True, False, False,
         True,  True, False],
       [False, False,  True, False,  True, False, False,  True, False,
        False,  True,  True]])

In [26]:
x[mutat_mask] = ~x[mutat_mask]

In [27]:
x

array([[ True, False, False, False, False, False,  True,  True,  True,
        False,  True,  True],
       [False,  True,  True, False, False,  True,  True, False, False,
         True, False,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True, False,
         True,  True, False]])

In [28]:
x = np.array([[1, 0, 1, 1, 1, 0],[0, 1, 0, 0, 1, 0]], dtype=bool)
x

array([[ True, False,  True,  True,  True, False],
       [False,  True, False, False,  True, False]])

In [29]:
flip_bit_mutate(x)

array([[False, False,  True,  True,  True, False],
       [False,  True, False, False,  True, False]])

In [31]:
x.astype(int)

array([[0, 0, 1, 1, 1, 0],
       [0, 1, 0, 0, 1, 0]])